In [1]:
!pip install --quiet pip transformers langchain

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langserve 0.2.3 requires langchain-core<0.3,>=0.1, but you have langchain-core 0.3.21 which is incompatible.
tensorflow 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.2 which is incompatible.
tf-nightly 2.19.0.dev20241202 requires numpy<2.2.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.5.1 which is incompatible.


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [ ]:
# from transformers import AutoModelForSequenceClassification, AutoTokenizer

# model_name = "jcblaise/roberta-tagalog-base"
# local_path = "./roberta-base"

# # Download the model and save it locally
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.save_pretrained(local_path)

# model = AutoModelForSequenceClassification.from_pretrained(model_name)
# model.save_pretrained(local_path)

In [2]:
# # Load model directly ( Download the model)

# model_name = "jcblaise/gpt2-tagalog"
# local_path = "./gpt2_tagalog"

# tokenizer_download = AutoTokenizer.from_pretrained(model_name)
# model_download = AutoModelForCausalLM.from_pretrained(model_name)

# model_download.save_pretrained(local_path)
# tokenizer_download.save_pretrained(local_path)

('./gpt2_tagalog/tokenizer_config.json',
 './gpt2_tagalog/special_tokens_map.json',
 './gpt2_tagalog/vocab.json',
 './gpt2_tagalog/merges.txt',
 './gpt2_tagalog/added_tokens.json',
 './gpt2_tagalog/tokenizer.json')

In [ ]:
model_path = "./gpt2_tagalog"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
prompt = "Kamusta, anong bali"
inputs = tokenizer(prompt, return_tensors="pt")
inputs

In [ ]:
# Generate response
outputs = model.generate(inputs.input_ids, max_length=int(len(prompt) + 1) , num_return_sequences=1)

# Decode the generated tokens
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
response

In [ ]:
generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
result = generator(prompt)
result

In [ ]:
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any, Dict, ClassVar
from transformers import AutoModelForCausalLM, AutoTokenizer  # Import these to use them

class CustomLLM(LLM):
    model_path: ClassVar[str] = "./gpt2_tagalog"  # Annotate as ClassVar
    model: Optional[AutoModelForCausalLM] = None  # Provide a type annotation
    tokenizer: Optional[AutoTokenizer] = None  # Provide a type annotation

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.model = AutoModelForCausalLM.from_pretrained(self.model_path)
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
        
    @property
    def _llm_type(self) -> str:
        return "custom"
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        return "This is a dummy LLM that always outputs the same prompt"
    
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {}

    def get_num_tokens(self, text: str) -> int:
        """Get the number of tokens present in the text."""
        return len(self.tokenizer.encode(text))


In [ ]:
llm = CustomLLM()

In [ ]:
# Use the model to generate text
prompt = "Kamusta, anong balita?"
response = llm(prompt)
print(response)

In [ ]:
llm.invoke(prompt)

In [ ]:
llm = CustomLLM()
prompt = "Complete the sentence: Kamusta, anong bali"
print(llm.invoke(prompt))

In [ ]:
from langchain.llms.base import LLM
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from typing import Optional, List, Mapping, Any, Dict, ClassVar

class CustomLLM(LLM):
    model_path: ClassVar[str] = "./gpt2_tagalog"  # Path to your Tagalog model
    model: Optional[AutoModelForCausalLM] = None
    tokenizer: Optional[AutoTokenizer] = None

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.model = AutoModelForCausalLM.from_pretrained(self.model_path)
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
        
    @property
    def _llm_type(self) -> str:
        return "custom"
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> List[str]:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")

        # Set up the generator to return limited word completions
        generator = pipeline(task="text-generation", model=self.model, tokenizer=self.tokenizer)
        results = generator(prompt, max_new_tokens=5, num_return_sequences=1)  # Generates 3 options

        # Extract and return the top three generated suggestions
        suggestions = result[0]['generated_text']
        return suggestions
    
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {}

    def get_num_tokens(self, text: str) -> int:
        """Get the number of tokens present in the text."""
        return len(self.tokenizer.encode(text))

In [ ]:
llm = CustomLLM()
prompt = "Complete the sentence: Kamusta, anong bali"
print(llm.invoke(prompt))

In [ ]:
from langchain.llms.base import LLM
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from typing import Optional, List, Mapping, Any, Dict, ClassVar

class CustomLLM(LLM):
    model_path: ClassVar[str] = "./gpt2_tagalog"  # Path to your Tagalog model
    model: Optional[AutoModelForCausalLM] = None
    tokenizer: Optional[AutoTokenizer] = None

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.model = AutoModelForCausalLM.from_pretrained(self.model_path)
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
        
    @property
    def _llm_type(self) -> str:
        return "custom"
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> List[str]:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")

        # Initialize the generator with beam search for better output diversity
        generator = pipeline(
            task="text-generation",
            model=self.model,
            tokenizer=self.tokenizer
        )
        result = generator(prompt, max_new_tokens=3, num_return_sequences=1, pad_token_id=generator.tokenizer.eos_token_id)
        suggestions = result[0]['generated_text']
        return suggestions
    
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {}

    def get_num_tokens(self, text: str) -> int:
        """Get the number of tokens present in the text."""
        return len(self.tokenizer.encode(text))

In [ ]:
# Define a prompt template for generating suggestions
template = "complete the last word of the sentence: {input_text}"
prompt_template = PromptTemplate(input_variables=["input_text"], template=template)

# Instantiate the Custom LLM
llm = CustomLLM()

# Define the input text
input_text = "Kamusta, anong bali"
prompt = prompt_template.format(input_text=input_text)

# Invoke the LLM with the prompt to get suggestions
suggestions = llm._call(prompt)

# Print out the suggestions
print("Suggestions:", suggestions)

In [ ]:
#10k